### FewshotPromptTemplate

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

In [2]:
from dotenv import dotenv_values
import os

env_vars = dotenv_values('../.env')

os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

In [3]:
model = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [9]:
examples = [
    {
        "question": "프랑스에 대해 무엇을 알고 있습니까?",
        "answer": """
        알고 있는 것:
        수도 : 파리
        언어 : French
        음식 : 와인, 치즈
        화폐 : 유로
        """,
    },
    {
        "question": "이탈리아에 대해 무엇을 알고 있습니까?",
        "answer": """
        알고 있는 것:
        수도 : 로마
        언어 : Italian
        음식 : 피자, 파스타
        화폐 : 유로
        """,
    },
        {
        "question": "그리스에 대해 무엇을 알고 있습니까?",
        "answer": """
        알고 있는 것:
        수도 : 아테네
        언어 : Greek
        음식 : 수블라키, 페타 치즈
        화폐 : 유로
        """,
    }
]

In [11]:
model.predict("프랑스에 대해 무엇을 알고 있습니까?")

프랑스는 유럽 대륙의 서쪽에 위치한 나라로, 파리를 수도로 가지고 있습니다. 프랑스는 세계적으로 유명한 문화와 예술, 요리, 패션 등으로 알려져 있으며, 에펠탑, 루브르 박물관, 베르사유 궁전 등 많은 유명한 관광지가 있습니다. 프랑스는 또한 세계에서 가장 방문자 수가 많은 나라 중 하나이며, 와인과 치즈 등의 다양한 음식 문화로도 유명합니다. 프랑스는 유럽 연합의 중요한 회원국이며, 세계 경제에서도 큰 역할을 합니다.

'프랑스는 유럽 대륙의 서쪽에 위치한 나라로, 파리를 수도로 가지고 있습니다. 프랑스는 세계적으로 유명한 문화와 예술, 요리, 패션 등으로 알려져 있으며, 에펠탑, 루브르 박물관, 베르사유 궁전 등 많은 유명한 관광지가 있습니다. 프랑스는 또한 세계에서 가장 방문자 수가 많은 나라 중 하나이며, 와인과 치즈 등의 다양한 음식 문화로도 유명합니다. 프랑스는 유럽 연합의 중요한 회원국이며, 세계 경제에서도 큰 역할을 합니다.'

- suffix는 형식화된 모든 예제 마지막에 나오는 내용 (사용자 질문)

In [14]:
examples_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=examples_prompt,
    examples=examples,
    suffix="Human: {country}에 대해 무엇을 알고 있습니까?",
    input_variables=["country"]
)

prompt.format(country="일본")

'Human: 프랑스에 대해 무엇을 알고 있습니까?\nAI: \n        알고 있는 것:\n        수도 : 파리\n        언어 : French\n        음식 : 와인, 치즈\n        화폐 : 유로\n        \n\nHuman: 이탈리아에 대해 무엇을 알고 있습니까?\nAI: \n        알고 있는 것:\n        수도 : 로마\n        언어 : Italian\n        음식 : 피자, 파스타\n        화폐 : 유로\n        \n\nHuman: 그리스에 대해 무엇을 알고 있습니까?\nAI: \n        알고 있는 것:\n        수도 : 아테네\n        언어 : Greek\n        음식 : 수블라키, 페타 치즈\n        화폐 : 유로\n        \n\nHuman: 일본에 대해 무엇을 알고 있습니까?'

In [15]:
chain = prompt | model
chain.invoke({
    "country": "일본"
})

AI: 
        알고 있는 것:
        수도 : 도쿄
        언어 : Japanese
        음식 : 초밥, 라면
        화폐 : 엔

AIMessageChunk(content='AI: \n        알고 있는 것:\n        수도 : 도쿄\n        언어 : Japanese\n        음식 : 초밥, 라면\n        화폐 : 엔')

### FewShotChatMessagePromptTemplate

In [4]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

In [5]:
examples = [
    {
        "country": "프랑스",
        "answer": """
        알고 있는 것:
        수도 : 파리
        언어 : French
        음식 : 와인, 치즈
        화폐 : 유로
        """,
    },
    {
        "country": "이탈리아",
        "answer": """
        알고 있는 것:
        수도 : 로마
        언어 : Italian
        음식 : 피자, 파스타
        화폐 : 유로
        """,
    },
        {
        "country": "그리스",
        "answer": """
        알고 있는 것:
        수도 : 아테네
        언어 : Greek
        음식 : 수블라키, 페타 치즈
        화폐 : 유로
        """,
    }
]

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("human", "{country}에 대해 무엇을 알고 있습니까?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 전문적인 지질학자 입니다. 당신은 추가적인 답변을 해야합니다."),
    example_prompt,
    ("human", "{country}에 대해 무엇을 알고 있습니까?")
])

chain = final_prompt | model
chain.invoke({"country": "중국"})


        알고 있는 것:
        수도 : 베이징
        언어 : 중국어
        음식 : 짜장면, 탕수육
        화폐 : 위안
        중국은 역사적으로 오랜 세월 동안 발전해온 문화와 전통을 가지고 있으며, 세계에서 가장 인구가 많은 국가 중 하나입니다. 중국은 또한 다양한 지리적 특성과 자연 장관을 가지고 있으며, 중국의 역사적인 유적지와 명소들은 많은 관광객들에게 인기가 있습니다. 또한 중국은 세계에서 가장 큰 경제 국가 중 하나로, 글로벌 경제에 큰 영향을 미치고 있습니다.

AIMessageChunk(content='\n        알고 있는 것:\n        수도 : 베이징\n        언어 : 중국어\n        음식 : 짜장면, 탕수육\n        화폐 : 위안\n        중국은 역사적으로 오랜 세월 동안 발전해온 문화와 전통을 가지고 있으며, 세계에서 가장 인구가 많은 국가 중 하나입니다. 중국은 또한 다양한 지리적 특성과 자연 장관을 가지고 있으며, 중국의 역사적인 유적지와 명소들은 많은 관광객들에게 인기가 있습니다. 또한 중국은 세계에서 가장 큰 경제 국가 중 하나로, 글로벌 경제에 큰 영향을 미치고 있습니다.')

## LengthBasedExampleSelector

In [11]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [12]:
examples = [
    {
        "question": "프랑스에 대해 무엇을 알고 있습니까?",
        "answer": """
        알고 있는 것:
        수도 : 파리
        언어 : French
        음식 : 와인, 치즈
        화폐 : 유로
        """,
    },
    {
        "question": "이탈리아에 대해 무엇을 알고 있습니까?",
        "answer": """
        알고 있는 것:
        수도 : 로마
        언어 : Italian
        음식 : 피자, 파스타
        화폐 : 유로
        """,
    },
        {
        "question": "그리스에 대해 무엇을 알고 있습니까?",
        "answer": """
        알고 있는 것:
        수도 : 아테네
        언어 : Greek
        음식 : 수블라키, 페타 치즈
        화폐 : 유로
        """,
    }
]

In [15]:
examples_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=examples_prompt,
    max_length=180
)

prompt = FewShotPromptTemplate(
    example_prompt=examples_prompt,
    example_selector=example_selector,
    suffix="Human: {country}에 대해 무엇을 알고 있습니까?",
    input_variables=["country"]
)

prompt.format(country="일본")

'Human: 프랑스에 대해 무엇을 알고 있습니까?\nAI: \n        알고 있는 것:\n        수도 : 파리\n        언어 : French\n        음식 : 와인, 치즈\n        화폐 : 유로\n        \n\nHuman: 이탈리아에 대해 무엇을 알고 있습니까?\nAI: \n        알고 있는 것:\n        수도 : 로마\n        언어 : Italian\n        음식 : 피자, 파스타\n        화폐 : 유로\n        \n\nHuman: 일본에 대해 무엇을 알고 있습니까?'

In [25]:
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

In [27]:
examples_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=examples_prompt,
    example_selector=example_selector,
    suffix="Human: {country}에 대해 무엇을 알고 있습니까?",
    input_variables=["country"]
)

prompt.format(country="일본")

'Human: 프랑스에 대해 무엇을 알고 있습니까?\nAI: \n        알고 있는 것:\n        수도 : 파리\n        언어 : French\n        음식 : 와인, 치즈\n        화폐 : 유로\n        \n\nHuman: 일본에 대해 무엇을 알고 있습니까?'